# RRab Phase Density Diagrams

## Imports 

In [1]:
import warnings

In [2]:
import numpy as np
import pandas as pd
import scipy as sp

In [3]:
from astroquery.gaia import Gaia

Created TAP+ (v1.0.1) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: False
	Port: 80
	SSL Port: 443


In [4]:
import bokeh
from bokeh.plotting import figure, output_file, show
from bokeh.models import Range1d, Ticker
from bokeh.layouts import gridplot, layout
import bokeh.palettes as palettes

In [5]:
from bokeh.plotting import output_notebook
output_notebook()

Loading BokehJS ...

In [6]:
import datashader as ds
import datashader.transfer_functions as tf
from datashader.bokeh_ext import InteractiveImage

In [7]:
from astropy.io import votable
from astropy.io import ascii

In [68]:
from time import time as timer

## Get a list of all SOS RRab stars

In [50]:
query = \
"""
select
    t1.source_id, t1.pf, t1.epoch_g, t1.peak_to_peak_g, t1.int_average_g as meanG, t2.epoch_photometry_url
from
    gaiadr2.vari_rrlyrae as t1
    inner join
    gaiadr2.gaia_source as t2
    on t1.source_id = t2.source_id
where
    t1.best_classification = 'RRab'
    and
    t2.phot_g_mean_mag <= 18
"""

In [51]:
job = Gaia.launch_job_async(query, dump_to_file=False)

Launched query: '
select
    t1.source_id, t1.pf, t1.epoch_g, t1.peak_to_peak_g, t1.int_average_g as meanG, t2.epoch_photometry_url
from
    gaiadr2.vari_rrlyrae as t1
    inner join
    gaiadr2.gaia_source as t2
    on t1.source_id = t2.source_id
where
    t1.best_classification = 'RRab'
    and
    t2.phot_g_mean_mag <= 18
'
Retrieving async. results...


Query finished.


In [52]:
RRabTable = job.get_results()

In [53]:
Nstars = len(RRabTable)
print("Number of sources: ", Nstars)

Number of sources:  49445


In [54]:
RRabTable[:3]

source_id,pf,epoch_g,peak_to_peak_g,meang,epoch_photometry_url
,d,d,mag,mag,
int64,float64,float64,float64,float64,object
5887646280147139072,0.5337003945080656,1736.5364324349166,0.7322525151480512,16.516070678205647,http://geadata.esac.esa.int/data-server/data?RETRIEVAL_TYPE=epoch_photometry&ID=5887646280147139072
5887054296218238336,0.4321570593237714,1703.2299657314775,0.3511027065782102,16.901611226965663,http://geadata.esac.esa.int/data-server/data?RETRIEVAL_TYPE=epoch_photometry&ID=5887054296218238336
5887438747319318016,0.7134263346636405,1702.5220956640428,0.6652171911329354,17.72227488995324,http://geadata.esac.esa.int/data-server/data?RETRIEVAL_TYPE=epoch_photometry&ID=5887438747319318016


## Download and process all RRab time series

Downloading 35K RRab GDR2 time series from the ESA Gaia archive took about 4h.

In [96]:
allPhases = np.array([], dtype=np.float)
allSignals = np.array([], dtype=np.float)

beginTime = timer()

for n in range(0, 35000):
    
    # Construct the download URL
    
    url = RRabTable['epoch_photometry_url'][n].decode("utf-8") 
    url += "&VALID_DATA=true&BAND=G"
    
    # Download the VOT table.
    
    try:
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            vot = votable.parse(url)
    except:
       continue

    table = vot.get_first_table()
    
    # Extract the time series
    
    time = table.array["time"].data
    Gmag = table.array["mag"].data
    
    # Construct the phase diagram. Time0 is the time of maximum magnitude.
    # Normalize the magnitudes 
    
    time0 = RRabTable['epoch_g'][n]
    period = RRabTable['pf'][n]
    meanG = RRabTable['meang'][n]
    P2Pamplitude = RRabTable['peak_to_peak_g'][n]
    
    phase = np.mod(time-time0, period)/period
    signal = (meanG - Gmag)/P2Pamplitude*2
    
    # Add them to our store
    
    allPhases = np.append(allPhases,   phase)
    allSignals = np.append(allSignals, signal)

endTime = timer()
print("Elapsed time: {0} min".format((endTime-beginTime)/60.))

Elapsed time: 52.81559654871623 min


Duplicate and shift the phase diagram, so that it ranges from 0 to 2 instead of 0 to 1.

In [98]:
results = pd.DataFrame({'phase' : np.concatenate([allPhases, allPhases+1]), 'signal' : np.concatenate([allSignals, allSignals])})

Make the density diagram

In [110]:
tools = "pan, box_zoom, undo, redo, save, reset"
fig = figure(width=850, height=400, x_range=(0,2), y_range=(-1.5, 1.5), tools=tools)
fig.xaxis.axis_label_text_font_size = "12pt"
fig.yaxis.axis_label_text_font_size = "12pt"
fig.xaxis.major_label_text_font_size = "12pt"
fig.yaxis.major_label_text_font_size = "12pt"

def image_callback(x_range, y_range, width, height):
    canvas = ds.Canvas(plot_width=width, plot_height=height, x_range=x_range, y_range=y_range)
    aggregated = canvas.points(results, 'phase', 'signal', agg=ds.reductions.count())
    image = tf.shade(aggregated, how='log', cmap=palettes.Plasma256)
    return image

InteractiveImage(fig, image_callback)